In [63]:
from collections import Counter
import random
import numpy as np
from tqdm.notebook import trange

In [64]:
PRINT_LEN = 50

In [65]:
def get_accuracy(pred, label):
    corrects = 0
    for i in range(len(pred)):
        if pred[i] == label[i]:
            corrects += 1
    acc = corrects / len(pred)
    print(f'Accuracy: {acc:.3f}')
    return acc

In [66]:
data_path = 'texts/WarAndPeace.txt'
with open(data_path, 'r') as f:
    corpus = f.read()
    
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
corpus = corpus.lower().replace('\n', ' ').replace('\t', ' ')
corpus = ''.join([x for x in corpus if x in alphabet])
corpus = ' '.join(corpus.split())
corpus[:PRINT_LEN]


'война и мир самый известный роман льва николаевича'

### 1. Базовый частотный метод

In [67]:
unigrams = Counter(corpus).most_common()
unigrams[:5]

[(' ', 103408), ('о', 61282), ('а', 45209), ('е', 42519), ('и', 35838)]

Зашифруем первые 10000 символов текста

In [68]:
test = corpus[:10000]

unique_chars = list(set(test))
unique_chars_shuffled = unique_chars.copy()
random.Random(42).shuffle(unique_chars_shuffled)

encoder_dict = dict(zip(unique_chars, unique_chars_shuffled))
trans_table = str.maketrans(encoder_dict)

test_encoded = test.translate(trans_table)
encoded_unigrams = Counter(test_encoded).most_common()
print('true:   ', test[:PRINT_LEN])
print('encoded:', test_encoded[:PRINT_LEN])

true:    война и мир самый известный роман льва николаевича
encoded: фьхмйншнэшзнцйэвхншуфац мвхнзьэймнтжфйнмшяьтйафшёй


Посмотрим, как меняется точность в зависимости от того, сколько самых частых символов заменяем

In [69]:
print('true:   ', test[:PRINT_LEN])
print('encoded:', test_encoded[:PRINT_LEN])
for i in range(len(encoded_unigrams)):
    decoder_dict = dict(zip([x[0] for x in encoded_unigrams[:i + 1]], [x[0] for x in unigrams[:i + 1]]))
    trans_table = str.maketrans(decoder_dict)
    test_decoded = test_encoded.translate(trans_table)
    if i % 5 == 0 or i == len(encoded_unigrams) - 1:
        print(f'top {i + 1:2} :', test_decoded[:PRINT_LEN], end='   ')
        get_accuracy(test, test_decoded);

true:    война и мир самый известный роман льва николаевича
encoded: фьхмйншнэшзнцйэвхншуфац мвхнзьэймнтжфйнмшяьтйафшёй
top  1 : фьхмй ш эшз цйэвх шуфац мвх зьэйм тжфй мшяьтйафшёй   Accuracy: 0.162
top  6 : фохие н энз цеэвх нуфац ивх зоэеи тжфе иняотеафнёе   Accuracy: 0.251
top 11 : сохие н энв леэвх нусалтивх воэеи ржсе иняореаснёе   Accuracy: 0.301
top 16 : сохие н днв ледвх нусалтивх водеи ржсе инкореаснёе   Accuracy: 0.348
top 21 : сохие н днв ледьх нгсалтиьх водеи рысе инкореаснзе   Accuracy: 0.348
top 26 : сойие н днв ледьй нгсалтиьй водеи рысе инкореаснзе   Accuracy: 0.375
top 31 : сойие н днв ледьй нгсалтиьй водеи рысе инкореаснзе   Accuracy: 0.379
top 34 : сойие н днв ледьй нгсалтиьй водеи рысе инкореаснзе   Accuracy: 0.382


Точность частотного подхода около 38%, хотя понять декодированный текст нельзя

### 2. Биграммный частотный метод

In [70]:
bigrams = Counter(corpus[i:i + 2] for i in range(len(corpus) - 1)).most_common()
print(f'bigrams: {len(bigrams)}')
bigrams[:5]

bigrams: 796


[('о ', 13316), ('и ', 11398), ('а ', 10596), ('е ', 10040), (' с', 9863)]

In [71]:
unique_chars = Counter(corpus[x:x+2] for x in range(len(corpus) - 1)).most_common() 
unique_chars = [x[0] for x in unique_chars]
unique_chars_shuffled = unique_chars.copy()
random.Random(42).shuffle(unique_chars_shuffled)

encoder_dict = dict(zip(unique_chars, unique_chars_shuffled))
encoded_bigrams = Counter(test_encoded[i:i + 2] for i in range(len(test_encoded) - 1)).most_common() 

In [72]:
print('true:    ', test[:PRINT_LEN])
print('encoded: ', test_encoded[:PRINT_LEN])
for i in range(len(encoded_bigrams)):
    decoder_dict = dict(zip([x[0] for x in encoded_bigrams[:i + 1]], [x[0] for x in bigrams[:i + 1]]))
    test_decoded = ''
    for j in range(0, len(test_encoded), 2):
        if test_encoded[j:j + 2] in decoder_dict:
            test_decoded += decoder_dict[test_encoded[j:j + 2]]
        else: 
            test_decoded += test_encoded[j:j + 2]
    if i % 50 == 0 or i == len(encoded_bigrams) - 1:
        print(f'top {i + 1:3} :', test_decoded[:PRINT_LEN], end='   ')
        get_accuracy(test, test_decoded);

true:     война и мир самый известный роман льва николаевича
encoded:  фьхмйншнэшзнцйэвхншуфац мвхнзьэймнтжфйнмшяьтйафшёй
top   1 : фьхмйншнэшзнцйэвхншуфац мвхнзьэймнтжфйнмшяьтйафшёй   Accuracy: 0.010
top  51 :  бхме и эшзнцйэвотшуу стмвотлоэймнтж т пшяьтйафшёй   Accuracy: 0.150
top 101 :  бхме и чтзнцйэвотшуу ст уотлоэйнясь т пшяк йатиёй   Accuracy: 0.170
top 151 :  бхме и чтзнцйэвотхоу ст уотлочанясь т пшяк шитины   Accuracy: 0.178
top 201 :  бр е и чтужцйэвотхоу ст уотлочанясь т пвнк шитины   Accuracy: 0.183
top 251 :  бр е и чтужебэвотхоу ст уотлочанясь т пвнк шитины   Accuracy: 0.185
top 301 :  бр е и чтужебаботхоу ст уотлочанясь т пвнк шитины   Accuracy: 0.186
top 351 :  бр е и чтужебаботхоу ст уотлочанясь т пвнк шитины   Accuracy: 0.186
top 401 :  бр е и чтужебаботхоу ст уотлочанясь т пвнк шитины   Accuracy: 0.187
top 451 :  бр е и чтужебаботхоу ст уотлочанясь т пвнк шитины   Accuracy: 0.187
top 501 :  бр е и чтужебаботхоу ст уотлочанясь т пвнк шитины   Accuracy: 0.187
top 505 :

Биграммный частотный метод дал точность 18%, что еще хуже, чем базовый подход

### 3. MCMC-сэмплированиe
Построим марковскую цепь для перестановки символов и найдем для нее стационарное распределение, соответствующее распределению n-грамм в русском языке
Воспользуемся алгоритмом Метрополиса-Гастингса, где коэффициент $a(text_{cur}, text_{new})=\frac{p*(text_{new})}{p*(text_{cur})}$ в качестве произвольного распределения возьмем равномерное, с поправкой на ассиметричность = 1.

Для вычислительной стабильности возмем логарифм от произведения. Нормировка на суммарное количество n-gramm не используется, т.к. с ней результаты хуже.

In [73]:
class MCMC:
    def __init__(self) -> None:
        self.vocab = None
        self.n_grams = None

    def fit(self, text, n_grams=2):
        self.vocab = list(set(text))
        self.n_grams = n_grams

        self.freq_dict = Counter(corpus[i:i + n_grams] for i in range(len(corpus) - n_grams + 1))
        for key in self.freq_dict.keys():
            self.freq_dict[key] = np.log(self.freq_dict[key])

    def change_text(self, text):
        token_1 = random.choice(self.vocab)[0]
        token_2 = random.choice(self.vocab)[0]
        encoder_dict = {
            token_1: token_2,
            token_2: token_1
            }
        trans_table = str.maketrans(encoder_dict)
        text_new = text.translate(trans_table)
        return text_new

    def get_score(self, text):
        score = 0
        for i in range(len(text) - self.n_grams + 1):
            cur_token = text[i:i + self.n_grams]
            score += self.freq_dict.get(cur_token, 0)
        return score
    
    def decode(self, text, num_iters=5000):
        best_text = cur_text = text
        best_score = cur_score = self.get_score(cur_text)

        iters = trange(num_iters, desc='Best score')
        for i in iters:
            new_text = self.change_text(cur_text)
            new_score = self.get_score(new_text)
            if new_score >= best_score:
                best_score = new_score
                best_text = new_text
                iters.set_description(f"Best score: {int(best_score)}")
            if new_score >= cur_score:
                cur_score = new_score
                cur_text = new_text
            elif np.log(random.random()) < (new_score - cur_score):
                cur_text = new_text
        return best_text

In [75]:
model = MCMC()
model.fit(corpus, 2)
test_decoded = model.decode(test_encoded)
print(test_encoded[:PRINT_LEN])
print(test_decoded[:PRINT_LEN])
get_accuracy(test_decoded, test);

Best score:   0%|          | 0/5000 [00:00<?, ?it/s]

фьхмйншнэшзнцйэвхншуфац мвхнзьэймнтжфйнмшяьтйафшёй
война и мир самый известный роман льва николаевича
Accuracy: 1.000


Алгоритм полностью расшифровал тестовый пример. Скорее всего распределение биграмм на 10 000 символов почти полностью соответствует распределению биграмм на всем корпусе текста.

P.S. алгоритм может сойтис не с первого раза.

### 4. Расшифровка сообщения

In [22]:
test_2 = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
test_2_dict = {elem: model.vocab[i] for i, elem in enumerate(set(test_2))}
trans_table = str.maketrans(test_2_dict)
test_2_ru = test_2.translate(trans_table)
test_2_decoded = model.decode(test_2_ru, 50000)
print(test_2)
print(test_2_decoded)

Best score:   0%|          | 0/50000 [00:00<?, ?it/s]

←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
если вы вимите норзальный или подти норзальный текст у этого сообщения который легко продитать скорее всего вы все смелали правильно и полудите заксизальный балл ча послемнее детвертое чамание курса хотя конедно я нидего не обещаж


После нескольких перезапусков алгоритма, удалось получить примерный перевод сообщения. Возможно для лучшей сходимости нужно обучить модель на большем количестве корпусов.

### 5. Сэмплирование с n-граммами

In [76]:
# 3-граммы
model = MCMC()
model.fit(corpus, 3)
test_decoded = model.decode(test_encoded)
print(test_encoded[:PRINT_LEN])
print(test_decoded[:PRINT_LEN])
get_accuracy(test_decoded, test);

Best score:   0%|          | 0/5000 [00:00<?, ?it/s]

фьхмйншнэшзнцйэвхншуфац мвхнзьэймнтжфйнмшяьтйафшёй
война и мир самый известный роман льва николаевича
Accuracy: 1.000


In [77]:
# 4-граммы
model = MCMC()
model.fit(corpus, 4)
test_decoded = model.decode(test_encoded)
print(test_encoded[:PRINT_LEN])
print(test_decoded[:PRINT_LEN])
get_accuracy(test_decoded, test);

Best score:   0%|          | 0/5000 [00:00<?, ?it/s]

фьхмйншнэшзнцйэвхншуфац мвхнзьэймнтжфйнмшяьтйафшёй
война и мир самый известный роман льва николаевича
Accuracy: 1.000


In [78]:
# 5-граммы
model = MCMC()
model.fit(corpus, 5)
test_decoded = model.decode(test_encoded, 50000)
print(test_encoded[:PRINT_LEN])
print(test_decoded[:PRINT_LEN])
get_accuracy(test_decoded, test);

Best score:   0%|          | 0/50000 [00:00<?, ?it/s]

фьхмйншнэшзнцйэвхншуфац мвхнзьэймнтжфйнмшяьтйафшёй
война и мир самый известный роман льва николаевича
Accuracy: 1.000


3-граммы,  4-граммы и 5-граммы так же отлично переводят текст, скорее всего это связано с большим размером зашифрованного текста (10000 символов).

### 6. Другие применения модели

Подозреваю, что модель можно использовать в медицине для поиска антибиотиков путем перестановки аминокислот (существующие антибиотики образуют стационарное распределение).